# 2D CSGA: An algebra for conic sections

Paper: https://link.springer.com/article/10.1007/s00006-018-0879-2.
Example taken from https://enkimute.github.io/ganja.js/examples/coffeeshop.html#csga2d_opns.

We start by initializing the algebra $\mathbb{R}_{5,3}$.

In [ ]:
%pip install -q kingdon anywidget==0.9.9 ipywidgets==8.1.3

In [ ]:
from kingdon import Algebra
from math import sin, cos, pi
from random import random
import timeit

alg = Algebra(5, 3)

Similar to CGA we need to make a Witt basis, but now we have three Witt pairs. The basis vectors $\mathbf{e}_1$ and $\mathbf{e}_2$ remain for the $(x, y)$ coordinates.

In [ ]:
e1, e2 = [alg.blades[key] for key in ['e1', 'e2']]
p1, p2, p3 = [alg.blades[key] for key in ['e3', 'e4', 'e5']]
n1, n2, n3 = [alg.blades[key] for key in ['e6', 'e7', 'e8']]
  
# infinity (i) and origin (o) : plus (p), minus (m), times (t). 
ip, im, it = [n1-p1,n2-p2,n3-p3]
op, om, ot = alg.scalar(e=0.5) * [n1+p1,n2+p2,n3+p3]

We then define the 'up' (C) function that takes a Euclidean point and casts it into $\mathbb{R}_{5,3}$. Moreover, we need to define normalization in such a way that the norm is always positive.

In [ ]:
def up(x,y):
    return op + x*e1 + y*e2 + 0.5*(x*x+y*y)*ip + 0.5*(x*x-y*y)*im + x*y*it

def normalized(x):
    return x / abs(x.sp(x).e)**0.5

We then generate 3 random points on a circle:

In [ ]:
thetas = [random()*2*pi for _ in range(3)]
P0, P1, P2 = [up(sin(theta), cos(theta)) for theta in thetas]
P0, P1, P2

And wedge them together into a circle:

In [ ]:
circle_opns = normalized(P0^P1^P2^im^it^om^ot)
circle_opns

Do a line too:

In [ ]:
line_opns = normalized(P0^P1^ip^im^it^om^ot)

Axis aligned Conic (with one point missing) (we'll animate that last point)

In [ ]:
conic_minus_one_point_opns = P0^P1^P2^it^om^ot

In [ ]:
def graph_func():
    time = timeit.default_timer()
    last_point = up( 0.7*sin(time),  0.7*cos(time) )
    # add that last point into the conic.
    conic_opns = normalized(conic_minus_one_point_opns ^ last_point)
    # The circle and line cut in a point-pair:
    pair = normalized(line_opns&circle_opns)
    
    return [
      0x00FF00,pair*2, 
      0xFF0000,circle_opns*10,
      0x0000FF,line_opns*10,
      0x000000,conic_opns*4,
      0xFF0000,last_point*100,
    ]

alg.graph(
    graph_func,
    animate=1,
    width='800px',
    height='600px',
    up=up,
)